In [1]:
from __future__ import print_function
from future_builtins import map, filter, zip

In [2]:
import tensorflow as tf
import keras
import coremltools
import numpy as np

for name, package in [('tensorflow',tf), ('keras',keras), ('coremltools',coremltools), ('numpy', np)]:
    try:
        print(name + ' v' + package.__version__)
    except AttributeError:
        print(name + ' v?')

Using TensorFlow backend.


tensorflow v1.4.0
keras v2.1.1
coremltools v?
numpy v1.13.3


In [3]:
# Load newline-delimited list of categories

# f = open('quickdraw-categories.txt','r')
f = open('50-categories.txt','r')
categories = [line.rstrip() for line in f]
f.close()

filename_from_category = lambda s: '../../quickdraw/'+s+'.npy'
filenames = list(map(filename_from_category, categories))

In [4]:
# Test that all files exist according to categories
# in quickdraw-categories.txt

import os.path

all_exist = True
for filename in filenames:
    if not os.path.isfile(filename):
        print('file `{}` does not exist'.format(filename))
        all_exist = False
if all_exist:
    print('All {} files found!'.format(len(categories)))

All 50 files found!


In [5]:
image_size = (28,28)
samples_per_category = 25000

In [6]:
def one_hot_vector(index):
    hot = np.zeros(len(categories),dtype=np.int8)
    hot[index] = 1
    return hot

def one_hot_array(index):
    is_column = False
    hot = np.zeros((1,len(categories)),dtype=np.int8)
    hot[0,index] = 1
    if is_column:
        return hot.T
    else:
        return hot

In [8]:
import random

def make_dataset():
    data = []
    target = []
    for category_index, (category, filename) in enumerate(zip(categories, filenames)):
        label_one_hot_encoded = one_hot_vector(category_index)
        category_images = np.load(filename)
        n_images = category_images.shape[0]
        for image in category_images[:samples_per_category]:
            data.append(image)
            target.append(label_one_hot_encoded)
    data = np.array(data)
    data = np.expand_dims(data, axis=1)
    target = np.array(target)
    print(data.shape)
    print(target.shape)
    return (data, target)

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Reshape, Dense, BatchNormalization, Dropout, \
                         Conv2D, MaxPooling2D, Activation, Flatten, Lambda

def _make_convolution_layers(model):
    model.add(
        Reshape(
            (1, input_shape[0], input_shape[1]), 
            input_shape=(1, input_shape[0]*input_shape[1])
        )
    )
    
    model.add(
        Lambda(lambda x: x/127.5-1)
    )

    for filters in n_filters:
        # Add a single convolution layer
        model.add(
            Conv2D(
                filters=filters,
                kernel_size=(3,3),
                padding='same',
                activation=layer_activation
            )
        )
        # Add batch normalization to the convolution layer
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
        # Pool the layer
        model.add(
            MaxPooling2D(pool_size=(2,2), data_format='channels_first')
        )

    model.add(Flatten())

def _make_dense_layers(model):
    for size in dense_sizes:
        model.add(
            Dense(size, activation=layer_activation)
        )
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
    model.add(
        Dense(output_size, activation=final_activation)
    )

def one_hot_output(y_hat_raw):
    # predict() outputs floats. We want one-hot in most cases
    y_hat = np.zeros(y_hat_raw.shape, dtype=np.int)
    y_hat[np.arange(y_hat_raw.shape[0]), y_hat_raw.argmax(1)] = 1
    return y_hat

In [10]:
data, target = make_dataset()

(1250000, 1, 784)
(1250000, 50)


In [11]:
import os.path
from keras.models import load_model
import json

def cache_fit(model_name, model, *args, **kwargs):
    archive_name = model_name+'_model.h5'
    history_name = model_name+'_history.json'
    archive_exists = os.path.isfile(archive_name)

    if not archive_exists:
        print('Model '+model_name+' not found in archive. Training new model.')
        hist = model.fit(*args, **kwargs)
        model.save(archive_name)
        with open(history_name, 'w') as f:
            json.dump(hist.history, f)
        return model
    else:
        print('Model found on disk. Reloading.')
        return load_model(archive_name)

In [14]:
input_shape=image_size
n_filters=[64,64,128,128]
dense_sizes=[128,128]
output_size=len(categories)

batch_normalize = True

layer_activation='relu'
final_activation='softmax'

loss='categorical_crossentropy'
optimizer='Nadam'
metrics=['categorical_accuracy']

#########

model = Sequential()

_make_convolution_layers(model)
_make_dense_layers(model)

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics
)

model = cache_fit(
    'cnn_50cat_25000img_8ep_morefilters', model, 
    x=data, y=target, 
    batch_size=32, epochs=8, 
    verbose=1, shuffle=True
)

Model cnn_50cat_25000img_8ep_morefilters not found in archive. Training new model.
Epoch 1/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.7689 - categorical_accuracy: 0.7925
Epoch 2/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.5813 - categorical_accuracy: 0.8441
Epoch 3/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.5368 - categorical_accuracy: 0.8559
Epoch 4/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.5117 - categorical_accuracy: 0.8624
Epoch 5/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.4954 - categorical_accuracy: 0.8669
Epoch 6/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.4830 - categorical_accuracy: 0.8702
Epoch 7/8
1250000/1250000 [==============================] - 240s 192us/step - loss: 0.4730 - categorical_accuracy: 0.8728
Epoch 8/8
1250000/1250000 [=============================

In [15]:
coreml_model = coremltools.converters.keras.convert(
    model, 
    input_names='drawing', 
    image_input_names='drawing', 
    class_labels='50-categories.txt'
)

ValueError: Keras layer '<class 'keras.layers.core.Lambda'>' not supported. 